In [3]:
import os
import numpy as np
import sys
import time
import matplotlib.pyplot as plt
from path import Path



In [ ]:
def read_csv_float_with_header(file1,list1):

    fileopen = open(file1,'rb')
    fileobject = csv.reader(fileopen)
   
   # get the header
    header = fileobject.next()  
   
   # read each line in the CSV, and convert the values to float before appending to list1
    for row in fileobject:
        float_line = []
        for subrow in row:
            float_line.append( float(subrow))
      
        list1.append( float_line)
      
    fileopen.close()   # close the file that we read from   
   
   return  header, list1
#********************
#**** end function reads a CSV file and returns a header, and a list that has been converted to float
#********************   

In [ ]:
def mainloop(driver_id, rdp_tolerance, Input_Path,route_order, previous_results):

    
    Input_Path =   DATA


    if (previous_results > 0):  # base of some previous results to speed up run time
        old_scores = []
        old_file = open("Results_"+str(previous_results)+"m/Driver_"+str(driver_id)+".csv",'rb')
        old_csv = csv.reader(old_file)
        for row in old_csv:
            route = int(row[1])
            num_matched = float(row[2])
            old_scores.append([route, num_matched])    
       #old_scores.sort()   


    start_time = time.time()
    
    
    list_of_paths = []   
    list_of_lengths = []
    
    
    list_of_routes = []
    

    if (route_order == 1):  # start in the beginning
        
        for cnt in range(1,201):
            list_of_routes.append(cnt)    
    
    elif (route_order ==2):  # go backwards
        for cnt in range(200,0,-1):#逆序排列并写入list_of_routes
            list_of_routes.append(cnt)        
    
    else:   # start in the middle
        for cnt in range(100,201):#后100个正常排序，前100个逆序排列
            list_of_routes.append(cnt)
        for cnt in range(99,0,-1):
            list_of_routes.append(cnt)    
    
    
    
    
    # read all the routes for this driver
    for cnt in list_of_routes:
    
       # initialize this path
        #代表文件系统路径
        path = Path(driver_id,cnt)   # start with driver 1, route 1
    
        input_coords=[] 
        file_name = os.path.join(Input_Path,str(driver_id),str(cnt) + ".csv")
        header, input_coords = read_csv_float_with_header(file_name,input_coords)#读入该司机的第i个里程作为一个列表
        path_array = np.array(input_coords)#将其转变成数组
       
        path.route = path_array
        path.time  = len(path.route)   # 1 second per data file，计算数组的长度，得到该里程的所有数据
        
        
        # only analyze this path if it is not within a 90 meter bound of the starting point
        max_value = np.amax(path.route)
        min_value = np.amin(path.route)
        if ( max_value < 90 and min_value > -90): #里程包含负值，即按绝对坐标系计算距离
            path.is_zero = 1   # this is a zero length route
            path.matched = 0   # the jitter is done differently
           
    

       # find the total distance along the route
        path.distance = path.get_route_distance(0, path.time)
        list_of_lengths.append(path.distance)#将整个里程的长度添加入列表


        if ( path.is_zero == 0) :    
           # get features on this path
           path.generate_features(rdp_tolerance)


In [24]:
!pip uninstall path.py-11.0.1

Skipping path.py-11.0.1 as it is not installed.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNC

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import precision_score,recall_score,f1_score,confusion_matrix,roc_curve
from sklearn.metrics import precision_recall_curve as prc
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import cross_val_predict as cvp
from sklearn.model_selection import cross_val_score as cvs

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import VotingClassifier as voting
from sklearn.ensemble import AdaBoostClassifier as Ada
from sklearn.ensemble import GradientBoostingClassifier as GBDT

data=pd.read_csv('C:\\Users\\hzg0601\\Desktop\\ensemble_learning\\SimPIP.csv')
data['Suspicion']-=1
X=data.iloc[:,2:len(data)]
y=data['Suspicion']

X_train,X_test,y_train,y_test=tts(X,y,test_size=0.2)

logit=LR()
logit.fit(X_train,y_train)
logit_predict=logit.predict(X_test)


#logit_cm=confusion_matrix(logit_predict,y_test)

#logit_mse=mse(logit_predict,y_test)

logit_precision=precision_score(logit_predict,y_test)
logit_recall=recall_score(logit_predict,y_test)
logit_f1=f1_score(logit_predict,y_test)

logit_cvs=cvs(logit,X,y,cv=5,scoring='accuracy')

y_scores=cvp(logit,X,y,cv=5,method='decision_function')

precision,recall,threshold=prc(y,y_scores)
fpr,tpr,threshold=roc_curve(y,y_scores)

logit_auc=auc(y,y_scores)

def plot_precision_recall_vs_threshold(precisions,recalls,thresholds):
    plt.plot(thresholds,precisions[:-1],'b--',label='precision')
    plt.plot(thresholds,recalls[:-1],'g--',label='recall')
    plt.xlabel('threshold')
    plt.legend()
    plt.ylim([0,1])
    
def plot_roc_curve(fpr,tpr,label=None):
    plt.plot(fpr,tpr,linewidth=2,label=label)
    plt.plot([0,1],[0,1],'k--')
    plt.axis([0,1,0,1])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    
#Gradient Tree Boosting (别名 GBM, GBRT, GBDT, MART)是一类很常用的集成学习算法